# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기

In [11]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='../chromedriver_win32/chromedriver.exe',options = options)

#driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("../dataset/2022/링크_크롤링/나무문화_링크_2022_9월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(20028, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/plumtreelee-/222888764192,2022 정조대왕 능행차 공동재현(서울 구간 축제소개/일정...,2022.09.30.
1,https://blog.naver.com/kimjihye525/222888763332,[전주야경] 덕진공원 & 카페한올 - 가을밤 산책코스,2022.09.30.
2,https://blog.naver.com/iland2279/222888762836,가을의 문지기 9월의 아라반,2022.09.30.
3,https://blog.naver.com/aceyun88/222888760839,"정남진 장흥 여행 특산물, 귀족호도박물관,귀족호도를 아시나요?",2022.09.30.
4,https://blog.naver.com/numain64/222888760244,"살기 싫어질때 가는 여행, 순천으로 가세요 (시즌 2)",2022.09.30.
...,...,...,...
20023,https://blog.naver.com/white7987/222863513830,영월 갈만한 곳 | 영월 관광센터 | 미디어 전시 | 문화도시 영월,2022.09.01.
20024,https://blog.naver.com/educationstar/222863308779,에너지 절약하면서 돈도 벌 수 있는 에코마일리지와...,2022.09.01.
20025,https://blog.naver.com/jang5071/222836825788,마음속의 마음 7장 4절,2022.09.01.
20026,https://blog.naver.com/soyoonchun1002/22286025...,명절에 제사를 지내야할까?,2022.09.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


20028


In [5]:
# df = pd.DataFrame(columns = ['url','본문']) # 처음 만들때를 위한 코드라 주석처리 하였음.
# df.to_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_9월.csv", index=False, encoding='UTF-8')
# df.to_excel("../나무문화/2022/나무문화_본문_2022_9월.xlsx", index=False, encoding='UTF-8')

In [5]:
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_9월.csv", encoding='UTF-8')
df

,url,본문
0,https://blog.naver.com/plumtreelee-/222888764192,#2022정조대왕능행차공동재현 #정조대왕능행차\n2022 정조대왕 능행차 공동재현\...
1,https://blog.naver.com/kimjihye525/222888763332,연꽃이 필때 와보고싶었지만\n더위에 약한 나는 이제서야 ...\n경기장튀김덕에 그래...
2,https://blog.naver.com/iland2279/222888762836,가을은 변화가 아름다울 수 있다는 것을\n가르쳐주는 계절이라고 합니다.\n여름의 뜨...
3,https://blog.naver.com/aceyun88/222888760839,"🚩정남진 장흥 여행 특산물, 귀족호도박물관,귀족호도를 아시나요?\n📌먼저 #정남진장..."
4,https://blog.naver.com/numain64/222888760244,우린 보성녹차를 왔다...\n드라마촬영지를 포기하고 여기를 왔다.\n이번에는 자연을...
...,...,...
10512,https://blog.naver.com/sje-life/222875534990,친환경소비문화형성\n미소나비기사\n지구별을 사랑하는 방법 100\n김분홍 미소나비기...
10513,https://blog.naver.com/dosim_senior/222875534485,인제에서 살아보기 (1)\n자기의 주거지를 떠나 객지를 돌아다니는 일을 “여행”이라...
10514,https://blog.naver.com/rin9041/222875529429,충청북도 옥천 장계관광지\n아름다운 대청호를 바라보며 거닐 수 있는 곳\n안녕하세요...
10515,https://blog.naver.com/skysunhome/222875529283,조만간\n조선을 만난 시간 프로젝트 시작!\n2022. 4. 9. ~ 11. 13....


In [7]:
len(df)

11602

In [9]:
url_df.iloc[11603]

url        https://blog.naver.com/venice87/222874141271
title    2023 아이테르 AITHER 전시 공모 (2022-09-14~2023-01-31)
date                                        2022.09.14.
Name: 11603, dtype: object

In [7]:
df.isnull().sum()

url    0
본문     1
dtype: int64

In [16]:
# # 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/fbi8864/221998085418"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

C:\Users\user\AppData\Local\Temp\ipykernel_5876\2516576624.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


In [10]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
        # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_9월.csv", index=False, encoding='UTF-8')
    #df.to_excel("./나무문화/2022/나무문화_본문_2022_9월.xlsx", index=False, encoding='UTF-8')
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 20%|█▉        | 1651/8426 [3:48:44<20:46:42, 11.04s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 20%|█▉        | 1652/8426 [3:48:53<19:13:34, 10.22s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 20%|█▉        | 1653/8426 [3:49:00<17:31:08,  9.31s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: Future

Alert Text: 게시물이 삭제되었거나 다른 페이지로 변경되었습니다.
Message: unexpected alert open: {Alert text : 게시물이 삭제되었거나 다른 페이지로 변경되었습니다.}
  (Session info: headless chrome=115.0.5790.171)

13708번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.


C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 25%|██▌       | 2107/8426 [4:50:10<12:58:56,  7.40s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 25%|██▌       | 2108/8426 [4:50:18<12:51:45,  7.33s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 25%|██▌       | 2109/8426 [4:50:25<12:51:29,  7.33s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15044\1621029842.py:35: Future

KeyboardInterrupt: 

In [ ]:
df